In [1]:
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
from sklearn.svm import SVC

In [3]:
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.model_selection import KFold

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#kmer1
fs1Train = ["/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_pos/combined/Kmer1/A.thaliana5289indep_pos_kmer_1.npy",
                   "/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_neg/concated result/concated kmer1/A.thaliana5289_neg_kmr1.npy"]
fs1Test = ["/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_pos_kmer_1.npy", 
                  "/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_neg_kmer_1.npy"]


#enac
fs2Train = ["/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_pos/combined/ENAC/A.thaliana5289indep_pos_ENAC.npy",
                   "/content/drive/My Drive/Pattern project training/pattern/data/converted/A.thaliana5289_neg/concated result/concated enac/A.thaliana5289_neg_ENAC.npy"]
fs2Test = ["/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_pos_ENAC.npy", 
                  "/content/drive/My Drive/Pattern project training/pattern/npy/A.thaliana1000indep_neg_enac.npy"]

In [7]:
kmer1_pos_5289 = fs1Train[0]
kmer1_neg_5289 = fs1Train[1]
kmer1_pos_1000 = fs1Test[0]
kmer1_neg_1000 = fs1Test[1]


enac_pos_5289 = fs2Train[0]
enac_neg_5289 = fs2Train[1]
enac_pos_1000 = fs2Test[0]
enac_neg_1000 = fs2Test[1]




x_train_pos = np.concatenate((np.load(kmer1_pos_5289),np.load(enac_pos_5289)),axis=1) 
x_train_neg = np.concatenate((np.load(kmer1_neg_5289),np.load(enac_neg_5289)),axis=1) 
x_test_pos = np.concatenate((np.load(kmer1_pos_1000),np.load(enac_pos_1000)),axis=1)  
x_test_neg = np.concatenate((np.load(kmer1_neg_1000),np.load(enac_neg_1000)),axis=1)  

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000) 

In [8]:
print(x_train_pos.shape)
print(x_train_neg.shape)
print(x_test_pos.shape)
print(x_test_neg.shape)

print(y_train_pos.shape)
print(y_train_neg.shape)
print(y_test_pos.shape)
print(y_test_neg.shape)

(5289, 152)
(5289, 152)
(1000, 152)
(1000, 152)
(5289,)
(5289,)
(1000,)
(1000,)


In [9]:
seed = 40
np.random.seed(seed)

In [10]:
x_training = np.concatenate((x_train_pos, x_train_neg), axis= 0)
y_training = np.concatenate((y_train_pos, y_train_neg))

In [11]:
x_test = np.concatenate((x_test_pos, x_test_neg))
y_test = np.concatenate((y_test_pos, y_test_neg))

In [12]:
print(x_test.shape)
print(y_test.shape)

(2000, 152)
(2000,)


In [13]:
print(x_training.shape)
print(y_training.shape)

(10578, 152)
(10578,)


In [14]:
kf = KFold(n_splits=5, random_state=seed)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [15]:
j=10
c_list=[]
c_test_avg =[]
c_train_avg = []
c_val_avg = [] 
while j <= 100:
  c = j
  sumTrain = sumVal = sumTest =  0
  print(f"SVC: kmer1+enac  |   c = {c}")
  clf = SVC(C=c,  random_state = seed)

  val_accuracy_list = []
  train_accuracy_list = []
  test_accuracy_list = []
  c = 1
  for train_index, val_index in kf.split(x_training):
      print("Fold : ",c)
      X_train, X_val = x_training[train_index], x_training[val_index]
      y_train, y_val = y_training[train_index], y_training[val_index]
      
      clf.fit(X_train, y_train)

      y_val_pred = clf.predict(X_val)
      val_accuracy = accuracy_score(y_val, y_val_pred)
      val_accuracy_list.append(val_accuracy)

      y_train_pred = clf.predict(X_train)
      train_accuracy = accuracy_score(y_train, y_train_pred)
      train_accuracy_list.append(train_accuracy)

      y_test_pred = clf.predict(x_test)
      test_accuracy = accuracy_score(y_test, y_test_pred)
      test_accuracy_list.append(test_accuracy)

      c += 1 

  print("done")
  for i in range(0, len(train_accuracy_list)):
      print("train_accuracy : ", train_accuracy_list[i], "\tVal accuracy : ", val_accuracy_list[i], "\tTest accuracy : ", test_accuracy_list[i])
      sumTrain = sumTrain + train_accuracy_list[i]
      sumVal = sumVal + val_accuracy_list[i]
      sumTest = sumTest + test_accuracy_list[i]
  sumTrain = sumTrain/5.0
  sumVal = sumVal/5.0
  sumTest = sumTest/5.0
  print("------------------------------------------------------------------------------------------------")
  print(f"Train accuracy(avg): {sumTrain} | Val accuracy(avg): {sumVal}  | Test accuracy(avg): {sumTest}")
  print("================================================================================================")
  c_list.append(j)
  c_train_avg.append(sumTrain)
  c_val_avg.append(sumVal)
  c_test_avg.append(sumTest)
  j = j+10

SVC: kmer1+enac  |   c = 10
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done
train_accuracy :  1.0 	Val accuracy :  0.31096408317580343 	Test accuracy :  0.5745
train_accuracy :  1.0 	Val accuracy :  0.7944234404536862 	Test accuracy :  0.5525
train_accuracy :  1.0 	Val accuracy :  0.9494328922495274 	Test accuracy :  0.578
train_accuracy :  1.0 	Val accuracy :  0.9754137115839243 	Test accuracy :  0.552
train_accuracy :  1.0 	Val accuracy :  0.8761229314420804 	Test accuracy :  0.4535
------------------------------------------------------------------------------------------------
Train accuracy(avg): 1.0 | Val accuracy(avg): 0.7812714117810043  | Test accuracy(avg): 0.5421
SVC: kmer1+enac  |   c = 20
Fold :  1
Fold :  2
Fold :  3
Fold :  4
Fold :  5
done
train_accuracy :  1.0 	Val accuracy :  0.31096408317580343 	Test accuracy :  0.5745
train_accuracy :  1.0 	Val accuracy :  0.7944234404536862 	Test accuracy :  0.5525
train_accuracy :  1.0 	Val accuracy :  0.9494328922495274 	Te

In [17]:
i=0  
print(" c      train                   val                    test")
print("===================================================================")
for x in c_list:
  print(f"{x}  -  {c_train_avg[i]} - {c_val_avg[i]} - {c_test_avg[i]}")
  print("--------------------------------------------------------------------")
  i=i+1

 c      train                   val                    test
10  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
20  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
30  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
40  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
50  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
60  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
70  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
80  -  1.0 - 0.7812714117810043 - 0.5421
--------------------------------------------------------------------
90  -  1.0 - 0.7812714117810043 - 0.5421
-------------------